In [1]:
import pandas as pd

# Define the bucket and file names
bucket_name = 'mimicivliza'  # Replace with your bucket name
mimic_iv_bhc = f's3://{bucket_name}/sample_data_100.csv'

# Load the files
mimic_iv_bhc_100 = pd.read_csv(mimic_iv_bhc)

# Display the data
mimic_iv_bhc_100.head(20)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,note_id,input,target,input_tokens,target_tokens
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...,This is a ___ yo F admitted to the hospital af...,1195,75
1,15638884-DS-4,<SEX> M <SERVICE> MEDICINE <ALLERGIES> Augment...,Mr. ___ is a ___ yo man with CAD with prior MI...,3496,1143
2,12435705-DS-14,<SEX> M <SERVICE> MEDICINE <ALLERGIES> ibuprof...,Mr. ___ is a ___ w/ Ph+ve ALL on dasatanib and...,5591,1098
3,12413577-DS-4,<SEX> F <SERVICE> OBSTETRICS/GYNECOLOGY <ALLER...,"On ___, Ms. ___ was admitted to the gynecology...",1119,221
4,17967161-DS-29,<SEX> M <SERVICE> SURGERY <ALLERGIES> lisinopr...,Mr. ___ underwent an angiogram on ___ which sh...,3307,439
5,16956007-DS-20,<SEX> M <SERVICE> SURGERY <ALLERGIES> Codeine ...,Mr. ___ is a ___ who underwent an exploratory ...,4168,1209
6,16919911-DS-15,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Penicil...,This is a ___ year old female with a recent di...,2059,208
7,15682570-DS-25,<SEX> M <SERVICE> MEDICINE <ALLERGIES> No Know...,"___ w/ h/o CAD ___ CABG LIMA to LAD, SVG to D1...",2215,451
8,12135369-DS-24,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Compazi...,Ms ___ is a ___ year old woman with a history ...,2132,416
9,11906321-DS-20,<SEX> M <SERVICE> NEUROSURGERY <ALLERGIES> Pat...,The patient was admitted to the neurosurgery s...,2347,316


In [2]:
# Install Hugging Face Transformers if not already installed
!pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import pandas as pd

# Step 1: Load the BioBART model and tokenizer
# Replace 'GanjinZero/biobart-large' with the desired BioBART model version
tokenizer = AutoTokenizer.from_pretrained("GanjinZero/biobart-large")
model = AutoModelForSeq2SeqLM.from_pretrained("GanjinZero/biobart-large")

tokenizer_config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

In [4]:
# Adjusting for BioBART

# Step 1: Tokenizer and Model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("GanjinZero/biobart-large")
model = AutoModelForSeq2SeqLM.from_pretrained("GanjinZero/biobart-large")

# Few-shot examples for prompt construction
few_shot_examples = [
    {
        "input": "<SEX> F <SERVICE> ONCOLOGY <CHIEF COMPLAINT> worsening back pain <HISTORY OF PRESENT ILLNESS> The patient is a 45-year-old female with a history of metastatic breast cancer presenting with worsening back pain over the last two weeks. Imaging revealed compression fractures in the thoracic spine. She reported increasing discomfort despite over-the-counter pain relievers. Neurological exam was unremarkable, and there were no signs of cord compression. Pain management and radiation oncology were consulted, and palliative radiation therapy was planned. The patient also discussed advanced care planning during her stay.",
        "target": "A 45-year-old female with metastatic breast cancer presented with worsening back pain. Imaging showed thoracic spine fractures, and she received palliative radiation therapy."
    },
    {
        "input": "<SEX> M <SERVICE> CARDIOLOGY <CHIEF COMPLAINT> chest pain <HISTORY OF PRESENT ILLNESS> A 55-year-old male presented with chest pain radiating to his left arm and jaw. He reported associated shortness of breath and nausea. Initial ECG showed ST-segment elevation in the inferior leads, and troponin levels were elevated, consistent with an acute myocardial infarction. The patient was emergently taken to the catheterization lab, where coronary angiography revealed 100% occlusion of the right coronary artery. A drug-eluting stent was successfully placed, and dual antiplatelet therapy was initiated. He was monitored in the cardiac care unit for 48 hours with no complications.",
        "target": "A 55-year-old male with a myocardial infarction underwent PCI with stenting of the right coronary artery. He was monitored and started on dual antiplatelet therapy."
    },
    {
        "input": "<SEX> M <SERVICE> NEUROSURGERY <CHIEF COMPLAINT> headache past ten days <HISTORY OF PRESENT ILLNESS> The patient is a 60-year-old male with a history of renal cell carcinoma, presenting with progressively worsening right-sided headaches over 10 days. Imaging revealed a large intracranial mass in the right temporoparietal lobe with associated hemorrhage, necrosis, and vasogenic edema. He was started on Decadron to reduce cerebral edema and transferred for neurosurgical evaluation. On hospital day 5, the patient underwent an image-guided right craniotomy for tumor resection. Postoperative MRI showed no significant residual tumor. He recovered without complications and was discharged home on a tapering dose of Decadron, with follow-up in the brain tumor clinic.",
        "target": "A 60-year-old male with an intracranial mass underwent a craniotomy for tumor resection. He recovered well and was discharged with follow-up."
    },
    {
        "input": "<SEX> F <SERVICE> GENERAL MEDICINE <CHIEF COMPLAINT> shortness of breath <HISTORY OF PRESENT ILLNESS> A 72-year-old female with a history of COPD presented with progressive shortness of breath over two weeks. Physical examination revealed decreased breath sounds and crackles in the left lower lobe. Imaging confirmed left lower lobe consolidation consistent with pneumonia. The patient was started on broad-spectrum antibiotics and oxygen therapy. Arterial blood gas analysis showed mild hypoxemia, which improved with oxygen supplementation. She remained afebrile throughout her stay and reported gradual improvement in symptoms. Discharge planning included antibiotics, a pulmonary follow-up, and instructions for home oxygen therapy.",
        "target": "A 72-year-old female with COPD was treated for pneumonia with antibiotics and oxygen therapy. She showed improvement and was discharged with follow-up."
    }
]

# Function to construct the prompt (unchanged)
def construct_few_shot_prompt(input_text):
    prompt = "You are a medical expert. Please summarize the following input concisely:\n\n"
    for example in few_shot_examples:
        prompt += f"Input: {example['input']}\nTarget: {example['target']}\n\n"
    prompt += f"Input: {input_text}\nSummary:"
    return prompt

# Step 2: Generate Summaries
generated_summaries = []

for input_text in tqdm(mimic_iv_bhc_100['input'], desc="Generating Summaries"):
    # Construct the prompt
    prompt = construct_few_shot_prompt(input_text)
    
    # Tokenize the prompt for BioBART
    inputs = tokenizer(
        prompt,
        max_length=512,  # BioBART's token limit
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Generate the summary
    generation_params = {
        "do_sample": True,
        "top_p": 0.8,
        "temperature": 0.1,
        "top_k": 40,
        "max_new_tokens": 300,
        "repetition_penalty": 1.1,
        "early_stopping": True
    }
    summary_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        **generation_params
    )
    
    # Decode the summary and append to the list
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(generated_summary)

# Step 3: Add Generated Summaries to the DataFrame
mimic_iv_bhc_100['generated_summary_biobart'] = generated_summaries

# Optional: Save the results
mimic_iv_bhc_100.to_csv("biobart_generated_summaries.csv", index=False)
print("Summaries generated and saved to 'biobart_generated_summaries.csv'")


Generating Summaries: 100%|██████████| 100/100 [38:33<00:00, 23.14s/it]

Summaries generated and saved to 'biobart_generated_summaries.csv'
